In [34]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split as sklearn_split
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
import pickle
plt.style.use("ggplot")

# 1 | Helper functions

In [35]:
def split_data(data, val_size=0.2, test_size=0.2, random_state=42):
    """
    Divide os dados em treino, validação e teste.
    """
    train_data, temp_data = sklearn_split(
        data, test_size=(val_size + test_size), random_state=random_state
    )
    val_data, test_data = sklearn_split(
        temp_data,
        test_size=(test_size / (val_size + test_size)),
        random_state=random_state,
    )
    return train_data, val_data, test_data

In [36]:
def prepare_surprise_data(data):
    """
    Prepara os dados no formato esperado pelo pacote surprise.
    """
    reader = Reader(rating_scale=(0, 1))
    return Dataset.load_from_df(
        data[["customer_id", "offer_id", "total_offer_completed"]], reader
    )

In [37]:
def train_funk_svd(train_data):
    """
    Treina o modelo FunkSVD usando os dados de treino.
    """
    # Divide os dados no formato surprise
    train_set, test_set = train_test_split(train_data, test_size=0.2, random_state=42)

    # Configura e treina o modelo
    model = SVD(n_factors=50, random_state=42)
    model.fit(train_set)

    # Avalia o modelo no conjunto de teste
    predictions = model.test(test_set)
    test_rmse = accuracy.rmse(predictions)

    return model, test_rmse

In [38]:
def evaluate_validation(model, val_data):
    """
    Faz as predições no conjunto de validação e retorna a métrica RMSE.
    """
    val_predictions = [
        model.predict(row["customer_id"], row["offer_id"], row["total_offer_completed"])
        for _, row in val_data.iterrows()
    ]

    val_rmse = accuracy.rmse(val_predictions, verbose=False)
    return val_predictions, val_rmse

In [39]:
def top_offers_per_customer(predictions):
    """
    Retorna as 3 ofertas com maior probabilidade de aceite para cada cliente no conjunto de validação.
    """
    pred_df = pd.DataFrame(
        predictions,
        columns=["customer_id", "offer_id", "actual", "predicted", "details"],
    )

    pred_df = pred_df.sort_values(
        by=["customer_id", "predicted"], ascending=[True, False]
    )

    return pred_df[["customer_id", "offer_id", "predicted"]]

In [40]:
def calcular_arpu_estimado(predictions, arpu_data, threshold=0.7):
    """
    Estima o ARPU com base nas predições do modelo e nos valores históricos de ARPU.

    Parâmetros:
        predictions: Predições geradas pelo modelo no conjunto de validação.
        arpu_data: DataFrame com o histórico de ARPU por oferta.
        threshold: Probabilidade mínima para considerar uma oferta aceita.

    Retorna:
        estimated_arpu: ARPU médio estimado para as ofertas recomendadas.
        impact_analysis: Análise do impacto potencial no negócio.
    """
    pred_df = pd.DataFrame(
        predictions,
        columns=["customer_id", "offer_id", "actual", "predicted", "details"],
    )

    # Filter the offer based on the threshold probabilities
    high_prob_offers = pred_df[pred_df["predicted"] >= threshold]

    # Merge with historical ARPU data
    high_prob_offers = high_prob_offers.merge(
        arpu_data[["offer_id", "arpu"]], on="offer_id", how="left"
    )

    # Get the estimated average ARPU weighted by predicted probabilities
    total_weighted_arpu = (
        high_prob_offers["predicted"] * high_prob_offers["arpu"]
    ).sum()
    total_customers = high_prob_offers["customer_id"].nunique()
    estimated_arpu = total_weighted_arpu / total_customers if total_customers > 0 else 0

    # Calcular impacto no ARPU por oferta
    arpu_by_offer = (
        high_prob_offers.groupby("offer_id")
        .apply(
            lambda x: (
                (x["predicted"] * x["arpu"]).sum() / x["customer_id"].nunique()
                if x["customer_id"].nunique() > 0
                else 0
            )
        )
        .reset_index(name="estimated_arpu_per_offer")
    )

    # Adicionar ARPU histórico às ofertas para comparação
    arpu_by_offer = arpu_by_offer.merge(
        arpu_data[["offer_id", "arpu"]], on="offer_id", how="left"
    )
    arpu_by_offer["impact_percentage"] = (
        (arpu_by_offer["estimated_arpu_per_offer"] - arpu_by_offer["arpu"])
        / arpu_by_offer["arpu"]
    ) * 100

    # Análise de impacto geral
    average_historical_arpu = arpu_data["arpu"].mean()
    improvement = (
        (estimated_arpu - average_historical_arpu) / average_historical_arpu * 100
    )

    impact_analysis = {
        "estimated_arpu": estimated_arpu,
        "average_historical_arpu": average_historical_arpu,
        "improvement_percentage": improvement,
        "arpu_by_offer": arpu_by_offer,
    }

    return estimated_arpu, impact_analysis, total_weighted_arpu

# 1 | Load data

In [41]:
hist_arpu = pd.read_csv("../data/final/historical_arpu.csv")
hist_arpu = hist_arpu.drop(columns="Unnamed: 0")
hist_arpu.head()

,offer_id,offer_id_mapping,total_amount,total_customers,arpu
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,6,3975808.70,144909,27.4366
1,2298d6c36e964ae4a3e7e9706d1fb8c2,7,7783803.46,154828,50.2739
2,2906b810c7d4411798c6938adc9daaa5,1,5433365.93,145660,37.3017
3,3f207df678b143eea3cee63160fa8bed,2,2980827.33,140564,21.2062
4,4d5c57ea9a6940dd891ad53e9dbe8da0,9,6534407.03,150664,43.3707


In [44]:
customers_offers_agg = pd.read_csv(
    "../data/final/customer_offers_agg.csv"
)

In [9]:
customers_offers_agg = customers_offers_agg.sort_values(by=["customer_id", "offer_id"])

# 2 | Prepare data to model

In [10]:
# Split data in train, test and validation, without prepare to surprise package
train_raw, val_raw, test_raw = split_data(customers_offers_agg)

In [11]:
# Prepara data to surprise package
surprise_train = prepare_surprise_data(train_raw)

# 3 | Modeling

In [ ]:
# Train model
model, test_rmse = train_funk_svd(surprise_train)
print(f"Test RMSE: {test_rmse:.4f}")

In [13]:
# Save the model as a pickle file
model_pkl_file = "../models/funksvd.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)

In [ ]:
# Evalualiation of the model in the validtion set
val_predictions, val_rmse = evaluate_validation(model, val_raw)
print(f"Validation RMSE: {val_rmse:.4f}")

In [ ]:
# Get the top 3 offers for a customer
best_offers = top_offers_per_customer(val_predictions)
print("\nMelhores Ofertas por Cliente:")
best_offers

O modelo implementado foi o funkSVD:

- A versão implementada do modelo foi a mais simples, visto que utilizou apenas os dados necessários, i.e., sem a adição de nenhuma variável latente/explicativa

- O modelo também não teve os seus parâmetros ajustados

- O funkSVD foi escolhido devido a facilidade de implementação e, também, pelas características do modelo (robusto quando aplicado em dados esparsos, escalável para grandes conjuntos de dados, permite regularização, etc). Ele é utilizado para prever a probabiliade de aceitação de ofertas não vistas por um cliente. O modelo utiliza apenas a matriz  de interações (```customer_id```, ```offer_id``` e ```total_offer_completed```)

Logo o modelo poderia ser melhorado das seguintes formas: 

- Adicionar a otimização dos hiperpârametros do modelo:
	•	n_factors: Dimensão dos fatores latentes.
	•	reg_all: Regularização para evitar overfitting.
	•	learning_rate: Taxa de aprendizado.

- Adicionar variáveis explicativas ao modelo:
    •	FunkSVD + Modelos Baseados em Features: Após gerar as predições do FunkSVD, incluímos outras features em um modelo secundário, como uma regressão ou árvore de decisão.
	•	Concatenação de Latent Factors e Features Adicionais: Após treinar o FunkSVD, usamos os fatores latentes gerados (usuário e item) como entrada para um modelo supervisionado, junto com as outras features.


Os dados foram separados em três: dados de treino, teste e validação do modelo. Os dados de validação serão utilizados para estimar o impacto do modelo no ARPU.

# 4 | ARPU

Abaixo mensuramos o impacto no ARPU

In [53]:
estimated_arpu, impact_analysis, total_weighted_arpu = calcular_arpu_estimado(
    val_predictions, hist_arpu, threshold=0.5
)

# Exibir os resultados
print(f"ARPU Estimado: {impact_analysis['estimated_arpu']:.2f}")
print(f"ARPU Histórico Médio: {impact_analysis['average_historical_arpu']:.2f}")
print(f"Melhoria Estimada: {impact_analysis['improvement_percentage']:.2f}%")

ARPU Estimado: 58.88
ARPU Histórico Médio: 46.36
Melhoria Estimada: 27.00%


/var/folders/m6/mv_nh0ks2n98r3d6kxb3tdh40000gn/T/ipykernel_5785/2271383863.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  high_prob_offers.groupby("offer_id")


In [ ]:
hist_arpu["arpu"].mean()

Para estimar o impacto geral no ARPU, será utilizado os resultados do modelo funkSVD e da abordagem de top 3 ofertas tem

In [48]:
# Top 3 offers with the best ARPU
n_noinfo_top_3 = 2174
arpu_estimated_top_3 = 65.77

In [49]:
arpu_ml = estimated_arpu
n_ml = customers_offers_agg.customer_id.nunique() - n_noinfo_top_3

In [51]:
arpu_total = (arpu_ml * n_ml + arpu_estimated_top_3 * n_noinfo_top_3) / (
    n_noinfo_top_3 + n_ml
)
arpu_total

59.75919380946227